# YOLOv8

In [2]:
import os
import argparse
import logging
import matplotlib.pyplot as plt
from ultralytics import YOLO
from IPython.display import display, Image
from PIL import Image
import pandas as pd
import numpy as np

## Download Images from Bucket

In [ ]:
import sys
sys.path.append('../src')
from data_utils import download_images_and_labels


# Bucket and directory
bucket_name = 'tree-counting-project'
prefix = 'Dataset 348-17-15/' # using Roboflow dataset with 348 train, 17 val and 15 test images
local_dir = '../Data/Dataset 348-17-15'

# Download images
download_images_and_labels(bucket_name, prefix, local_dir)

## Installing YOLO

In [1]:
!pip install ultralytics==8.0.196

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.196 🚀 Python-3.10.14 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 14918MiB)
Setup complete ✅ (1 CPUs, 3.6 GB RAM, 41.2/98.2 GB disk)


## Training

In [2]:
# Function for logging - irrelevant as YOLO has built-in logging for training
def setup_logging(log_dir):
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)
    logging.basicConfig(filename=os.path.join(log_dir, 'log.txt'),
                        filemode='a',
                        format='%(message)s',
                        level=logging.DEBUG)
    logger = logging.getLogger()
    return logger


In [3]:
# function to plot training and val loss curves
def plot_metrics(log_dir, metrics):
    epochs = range(len(metrics['train_loss']))
    
    plt.figure()
    plt.plot(epochs, metrics['train_loss'], label='Train Loss')
    plt.plot(epochs, metrics['val_loss'], label='Val Loss')
    plt.title('Loss Curves')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig(os.path.join(log_dir, 'loss_curve.png'))
    plt.close()
    
    plt.figure()
    plt.plot(epochs, metrics['train_map50'], label='Train mAP50')
    plt.plot(epochs, metrics['val_map50'], label='Val mAP50')
    plt.title('mAP50 Curves')
    plt.xlabel('Epochs')
    plt.ylabel('mAP50')
    plt.legend()
    plt.savefig(os.path.join(log_dir, 'map50_curve.png'))
    plt.close()


In [ ]:

def train_yolo(model_name, data_path, total_epochs, img_size, log_dir):
    # Load model
    model = YOLO(model_name)

    # Initialize metrics dictionary
    metrics = {'train_loss': [], 'val_loss': [], 'train_map50': [], 'val_map50': []}
    
    # Setup logging
    logger = setup_logging(log_dir)

    # Train model for all epochs
    results = model.train(
        data=data_path,
        epochs=total_epochs,
        imgsz=img_size,
        save=True,
        plots=True
    )
    
#     for epoch in range(total_epochs):
#         epoch_results = results.epoch_results[epoch]
        
#         # Extract metrics for the epoch
#         train_loss = epoch_results['train']['loss'][0]  # assuming loss is stored in the first index
#         val_loss = epoch_results['val']['loss'][0]  # assuming loss is stored in the first index
#         train_map50 = epoch_results['train']['map50'][0]  # mAP at IoU=0.50 for training
#         val_map50 = epoch_results['val']['map50'][0]  # mAP at IoU=0.50 for validation

#         # Store metrics
#         metrics['train_loss'].append(train_loss)
#         metrics['val_loss'].append(val_loss)
#         metrics['train_map50'].append(train_map50)
#         metrics['val_map50'].append(val_map50)

#         # Log metrics
#         log_entry = {
#             'epoch': epoch,
#             'train_loss': train_loss,
#             'val_loss': val_loss,
#             'train_map50': train_map50,
#             'val_map50': val_map50,
#             'n_parameters': model.model.n_params()
#         }
#         logger.info(json.dumps(log_entry))

#         # Save checkpoint
#         model.save(f'{log_dir}/checkpoint_epoch_{epoch}.pt')

#     # Plot metrics
#     plot_metrics(log_dir, metrics)

In [5]:
# argument parsing
def parse_args():
    parser = argparse.ArgumentParser(description="Train YOLOv8 model.")
    parser.add_argument('--model', type=str, default='yolov8s.pt', help='Pre-trained model path')
    parser.add_argument('--data', type=str, required=True, help='Path to dataset')
    parser.add_argument('--epochs', type=int, default=25, help='Number of epochs to train')
    parser.add_argument('--img_size', type=int, default=800, help='Image size for training')
    parser.add_argument('--log_dir', type=str, default='./logs', help='Directory to save logs and plots')
    return parser.parse_args()

def parse_args_notebook():
    args = argparse.Namespace(
        model='yolov8s.pt',
        data='/home/jupyter/ee_tree_counting/Data/Dataset 348-17-15/data.yaml',
        epochs=100,
        img_size=640,
        batch=16,
        log_dir='./logs'
    )
    return args


In [6]:
args = parse_args_notebook()
train_yolo(args.model, args.data, args.epochs, args.img_size, args.log_dir)

New https://pypi.org/project/ultralytics/8.2.49 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.48 🚀 Python-3.10.14 torch-1.13.1+cu117 CUDA:0 (Tesla T4, 14918MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/home/jupyter/ee_tree_counting/Data/Dataset 348-17-15/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False

train: Scanning /home/jupyter/ee_tree_counting/Data/Dataset 348-17-15/train/labels.cache... 348 images, 0 backgrounds, 0 corrupt: 100%|██████████| 348/348 [00:00<?, ?it/s]

WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 368, len(boxes) = 29937. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.



val: Scanning /home/jupyter/ee_tree_counting/Data/Dataset 348-17-15/valid/labels.cache... 17 images, 0 backgrounds, 0 corrupt: 100%|██████████| 17/17 [00:00<?, ?it/s]


Plotting labels to runs/detect/train/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 4 dataloader workers
Logging results to runs/detect/train
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all         17       1130      0.837      0.823      0.849      0.394



100 epochs completed in 0.207 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 22.5MB
Optimizer stripped from runs/detect/train/weights/best.pt, 22.5MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.2.48 🚀 Python-3.10.14 torch-1.13.1+cu117 CUDA:0 (Tesla T4, 14918MiB)
Model summary (fused): 168 layers, 11125971 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  3.36it/s]


                   all         17       1130      0.851      0.829      0.871      0.418
Speed: 0.2ms preprocess, 4.2ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to runs/detect/train


AttributeError: 'DetMetrics' object has no attribute 'epoch_results'. See valid attributes below.

    This class is a utility class for computing detection metrics such as precision, recall, and mean average precision
    (mAP) of an object detection model.

    Args:
        save_dir (Path): A path to the directory where the output plots will be saved. Defaults to current directory.
        plot (bool): A flag that indicates whether to plot precision-recall curves for each class. Defaults to False.
        on_plot (func): An optional callback to pass plots path and data when they are rendered. Defaults to None.
        names (tuple of str): A tuple of strings that represents the names of the classes. Defaults to an empty tuple.

    Attributes:
        save_dir (Path): A path to the directory where the output plots will be saved.
        plot (bool): A flag that indicates whether to plot the precision-recall curves for each class.
        on_plot (func): An optional callback to pass 

# Inference on Test Set

In [34]:
# Load our trained YOLO model
model = YOLO("/home/jupyter/ee_tree_counting/Notebooks/runs/detect/train/weights/best.pt")  

# Creating directory to store results
test_images_dir = '/home/jupyter/ee_tree_counting/Data/Dataset 348-17-15/test/images'
output_dir = '/home/jupyter/ee_tree_counting/Data/Dataset 348-17-15/test/Model Inference'
os.makedirs(output_dir, exist_ok=True)

# List of all test images
test_images = [os.path.join(test_images_dir, img) for img in os.listdir(test_images_dir) if img.endswith('.jpg')]
test_images.sort()

# Run inference on all test images
results = model(test_images)  

# Visualize and save the results
for i, r in enumerate(results):
    # Plot results image
    im_bgr = r.plot()  # BGR-order numpy array
    im_rgb = Image.fromarray(im_bgr[..., ::-1])  # RGB-order PIL image

    # Show results to screen (in supported environments)
    # r.show()

    # Save results to disk
    im_rgb.save(os.path.join(output_dir, f"results_{i}.jpg"))



0: 640x640 48 trees, 10.4ms
1: 640x640 79 trees, 10.4ms
2: 640x640 113 trees, 10.4ms
3: 640x640 113 trees, 10.4ms
4: 640x640 145 trees, 10.4ms
5: 640x640 24 trees, 10.4ms
6: 640x640 82 trees, 10.4ms
7: 640x640 87 trees, 10.4ms
8: 640x640 87 trees, 10.4ms
9: 640x640 84 trees, 10.4ms
10: 640x640 155 trees, 10.4ms
11: 640x640 122 trees, 10.4ms
12: 640x640 73 trees, 10.4ms
13: 640x640 137 trees, 10.4ms
14: 640x640 137 trees, 10.4ms
Speed: 1.5ms preprocess, 10.4ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 640)


# Compare with Ground Truth Count

In [37]:
label_dir = '/home/jupyter/ee_tree_counting/Data/Dataset 348-17-15/test/labels'

# List of labels
labels = [os.path.join(label_dir, img) for img in os.listdir(label_dir) if img.endswith('.txt')]
labels.sort()

for idx, label in enumerate(labels):
    file = pd.read_csv(label, delimiter=' ', header=None)
    print(f'Ground Truth {idx}: {len(file)} trees')


Ground Truth 0: 48 trees
Ground Truth 1: 69 trees
Ground Truth 2: 114 trees
Ground Truth 3: 114 trees
Ground Truth 4: 97 trees
Ground Truth 5: 20 trees
Ground Truth 6: 72 trees
Ground Truth 7: 89 trees
Ground Truth 8: 89 trees
Ground Truth 9: 77 trees
Ground Truth 10: 148 trees
Ground Truth 11: 112 trees
Ground Truth 12: 72 trees
Ground Truth 13: 130 trees
Ground Truth 14: 130 trees


# Calculating MAE and MSE 

In [4]:
predictions = np.array([48, 79, 113, 113, 145, 24, 82, 87, 87, 84, 155, 122, 73, 137, 137])
ground_truth = np.array([48, 69, 114, 114, 97, 20, 72, 89, 89, 77, 148, 112, 72, 130, 130])

# Calculating MAE
mae = np.mean(np.abs(predictions - ground_truth))

# Calculating MSE
mse = np.mean((predictions - ground_truth) ** 2)

mae, mse


(7.8, 188.46666666666667)

# Inference on Random Satellite Image

In [10]:
# Load our trained YOLO model
model = YOLO("/home/jupyter/ee_tree_counting/Notebooks/runs/detect/train/weights/best.pt")  

# Creating directory to store results
sat_images_dir = '/home/jupyter/ee_tree_counting/Sample Images'
os.makedirs(sat_images_dir, exist_ok=True)

# List of all test images
sat_imgs = [os.path.join(sat_images_dir, img) for img in os.listdir(sat_images_dir) if img.endswith('.png') or img.endswith('.jpg') or img.endswith('.tif')]
sat_imgs.sort()

# Run inference on all test images
results = model(sat_imgs)  

# Visualize and save the results
for i, r in enumerate(results):
    # Plot results image
    im_bgr = r.plot()  # BGR-order numpy array
    im_rgb = Image.fromarray(im_bgr[..., ::-1])  # RGB-order PIL image

    # Show results to screen (in supported environments)
    # r.show()

    # Save results to disk
    im_rgb.save(os.path.join(sat_images_dir, f"results_{i}.jpg"))


0: 640x640 99 trees, 16.5ms
1: 640x640 19 trees, 16.5ms
2: 640x640 10 trees, 16.5ms
Speed: 68.6ms preprocess, 16.5ms inference, 69.1ms postprocess per image at shape (1, 3, 640, 640)


# Zero-shot YOLO inference (baseline)

In [12]:
model = YOLO("/home/jupyter/ee_tree_counting/Notebooks/yolov8s.pt")  

# Creating directory to store results
test_images_dir = '/home/jupyter/ee_tree_counting/Data/Dataset 348-17-15/test/images'
output_dir = '/home/jupyter/ee_tree_counting/Data/Dataset 348-17-15/test/Model Inference Baseline'
os.makedirs(output_dir, exist_ok=True)

# List of all test images
test_images = [os.path.join(test_images_dir, img) for img in os.listdir(test_images_dir) if img.endswith('.jpg')]
test_images.sort()

# Run inference on all test images
results = model(test_images)  

# Visualize and save the results
for i, r in enumerate(results):
    # Plot results image
    im_bgr = r.plot()  # BGR-order numpy array
    im_rgb = Image.fromarray(im_bgr[..., ::-1])  # RGB-order PIL image

    # Save results to disk
    im_rgb.save(os.path.join(output_dir, f"results_{i}.jpg"))


0: 640x640 1 truck, 10.7ms
1: 640x640 (no detections), 10.7ms
2: 640x640 (no detections), 10.7ms
3: 640x640 (no detections), 10.7ms
4: 640x640 (no detections), 10.7ms
5: 640x640 1 cow, 10.7ms
6: 640x640 (no detections), 10.7ms
7: 640x640 (no detections), 10.7ms
8: 640x640 (no detections), 10.7ms
9: 640x640 1 carrot, 10.7ms
10: 640x640 (no detections), 10.7ms
11: 640x640 (no detections), 10.7ms
12: 640x640 1 refrigerator, 10.7ms
13: 640x640 (no detections), 10.7ms
14: 640x640 (no detections), 10.7ms
Speed: 2.1ms preprocess, 10.7ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 640)
